# Feature engineering demonstration

---
## Load data from CSV

In [ ]:
!ls

In [1]:
# Introduce imports which we will use next
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)

In [2]:
df = pd.read_csv ('deutsche-boerse-xetra-pds_week39.csv')

---
## Explore the data

In [ ]:
df.info ()

In [ ]:
df.sample (5)

In [ ]:
len(df.ISIN.unique())

In [ ]:
len(df.Mnemonic.unique())

In [ ]:
len(df.SecurityDesc.unique())

In [3]:
df.SecurityType.unique()

array(['ETF', 'Common stock', 'ETC', 'Other', 'ETN'], dtype=object)

In [5]:
df.SecurityType.value_counts ()

Common stock    286933
ETF              33960
ETC               2170
Other              745
ETN                134
Name: SecurityType, dtype: int64

In [ ]:
df.Currency.unique()

In [6]:
df.Currency.value_counts()

EUR    323919
USD        23
Name: Currency, dtype: int64

In [ ]:
len(df.SecurityID.unique())

---
## Clean the data 

In [8]:
df['dt_date'] = df.Date.astype(np.datetime64)

In [9]:
df['dt_time'] = (df.Date +' '+ df.Time).astype(np.datetime64)

In [10]:
df.SecurityType = df.SecurityType.astype('category')

In [11]:
df.drop(columns=['Currency', 'SecurityID','ISIN','SecurityDesc'], inplace=True)

In [ ]:
df.info ()

In [ ]:
df.sample(5)

---
## Engineer a feature set

In [12]:
# Mark data points with trades outside of opening and closing hours
date = '2017-10-16'
time_fmt = "%Y-%m-%d%H:%M"
opening_hours = datetime.strptime(date +"08:00", time_fmt)
closing_hours = datetime.strptime(date +"20:00", time_fmt)

df['in_trading_hours'] = 0
df.loc[(df.dt_time > opening_hours) & (df.dt_time <= closing_hours), 'in_trading_hours'] = 1

In [13]:
df['pct_change'] = (df.EndPrice - df.StartPrice)/df.StartPrice

In [14]:
df['day_of_week'] = df.dt_date.dt.weekday

In [15]:
df['min_of_day'] = df.dt_time.dt.hour*60 + df.dt_time.dt.minute

In [16]:
minutes_in_day = 24*60

df['sin_time'] = np.sin(2*np.pi*df.min_of_day/minutes_in_day)
df['cos_time'] = np.cos(2*np.pi*df.min_of_day/minutes_in_day)

In [17]:
df['movement'] = 0
df.loc[(df['pct_change'] > 0), 'movement'] = 1
df.loc[(df['pct_change'] < 0), 'movement'] = -1
df['next_movement'] = 0
df['next_movement'] = df.movement.shift (-1)

## Explore the engineered data set

In [18]:
df.sample (5)

,Mnemonic,SecurityType,Date,Time,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,dt_date,dt_time,in_trading_hours,pct_change,day_of_week,min_of_day,sin_time,cos_time,movement,next_movement
320819,OSR,Common stock,2017-10-20,14:40,63.490,63.490,63.490,63.490,456,2,2017-10-20,2017-10-20 14:40:00,0,0.000000,4,880,-0.642788,-0.766044,0,0.0
247027,KCO,Common stock,2017-10-19,08:23,10.535,10.535,10.535,10.535,421,3,2017-10-19,2017-10-19 08:23:00,0,0.000000,3,503,0.811574,-0.584250,0,-1.0
264587,G1A,Common stock,2017-10-20,08:41,40.425,40.425,40.360,40.390,2347,18,2017-10-20,2017-10-20 08:41:00,0,-0.000866,4,521,0.763232,-0.646124,-1,0.0
113037,ARL,Common stock,2017-10-17,09:05,35.130,35.150,35.130,35.150,332,3,2017-10-17,2017-10-17 09:05:00,0,0.000569,1,545,0.691513,-0.722364,1,0.0
319860,LINU,Common stock,2017-10-20,14:33,176.950,177.050,176.950,177.050,288,5,2017-10-20,2017-10-20 14:33:00,0,0.000565,4,873,-0.619094,-0.785317,1,0.0


In [19]:
df.next_movement.value_counts()

 0.0    186381
-1.0     69824
 1.0     67736
Name: next_movement, dtype: int64

In [20]:
df.corr()[['next_movement']]

,next_movement
StartPrice,0.000433
MaxPrice,0.000437
MinPrice,0.000440
EndPrice,0.000444
TradedVolume,-0.001360
NumberOfTrades,0.000666
in_trading_hours,0.003202
pct_change,0.010498
day_of_week,-0.003230
min_of_day,0.005476


In [21]:
pd.options.display.max_columns = 35
df.corr ()

,StartPrice,MaxPrice,MinPrice,EndPrice,TradedVolume,NumberOfTrades,in_trading_hours,pct_change,day_of_week,min_of_day,sin_time,cos_time,movement,next_movement
StartPrice,1.000000,1.000000,0.999999,0.999999,-0.088841,0.005808,0.002204,0.001392,0.000455,-0.001811,0.001384,-0.002420,0.000100,0.000433
MaxPrice,1.000000,1.000000,0.999999,1.000000,-0.088822,0.005983,0.002184,0.001631,0.000468,-0.001834,0.001403,-0.002377,0.000280,0.000437
MinPrice,0.999999,0.999999,1.000000,1.000000,-0.088864,0.005601,0.002223,0.001768,0.000440,-0.001784,0.001360,-0.002468,0.000300,0.000440
EndPrice,0.999999,1.000000,1.000000,1.000000,-0.088844,0.005787,0.002204,0.001976,0.000453,-0.001806,0.001379,-0.002425,0.000465,0.000444
TradedVolume,-0.088841,-0.088822,-0.088864,-0.088844,1.000000,0.345626,-0.016217,-0.021525,0.017288,-0.017295,0.014958,0.038552,-0.003111,-0.001360
NumberOfTrades,0.005808,0.005983,0.005601,0.005787,0.345626,1.000000,-0.048057,-0.030841,0.054896,-0.034875,0.028009,0.096449,-0.008283,0.000666
in_trading_hours,0.002204,0.002184,0.002223,0.002204,-0.016217,-0.048057,1.000000,0.003370,-0.652452,0.094328,-0.084972,-0.122648,0.003190,0.003202
pct_change,0.001392,0.001631,0.001768,0.001976,-0.021525,-0.030841,0.003370,1.000000,-0.002443,0.003106,-0.002627,-0.003353,0.424142,0.010498
day_of_week,0.000455,0.000468,0.000440,0.000453,0.017288,0.054896,-0.652452,-0.002443,1.000000,0.002365,-0.002150,-0.007327,-0.003226,-0.003230
min_of_day,-0.001811,-0.001834,-0.001784,-0.001806,-0.017295,-0.034875,0.094328,0.003106,0.002365,1.000000,-0.996917,-0.514232,0.005451,0.005476


---
## Quick test using sklearn

In [28]:
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(subsample=0.3, max_depth=12, verbose=1)

In [29]:
from sklearn.model_selection import train_test_split

data_X = pd.get_dummies(df.drop(columns=['Mnemonic', 'Date', 'Time', 'dt_date', 'dt_time', 'next_movement'], axis = 1)) 
data_Y = df.next_movement
data_Y.fillna(0, inplace=True)
train_X, test_X, train_Y, test_Y = train_test_split (data_X, data_Y, test_size=0.2)

sample_weights = np.ones(train_X.shape[0])

def report (i, estimator, local_vals):
    print ("iteration {}: complete".format (i))

In [ ]:
gbc.fit(train_X.values, train_Y.values, sample_weight=sample_weights, monitor=report)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1       77559.5461        2694.7103          188.69m
iteration 0: complete
         2       76535.9375        2184.7404          150.62m
iteration 1: complete
         3       75521.0581        1721.7744          134.19m
iteration 2: complete


In [ ]:
pred = gbc.predict(test_X)

In [ ]:
pred[:5]

In [ ]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
from sklearn.metrics import confusion_matrix
cfm = confusion_matrix(test_Y, pred)
cfm

In [ ]:
plot_confusion_matrix (cfm, classes=['down', 'stay', 'up'], normalize=True)

In [ ]:
!ls -lh work/ddata.csv

In [ ]:
!cp work/ddata.csv deutsche-boerse-xetra-pds_week39.csv